In [1]:
# external api
import fundamentalanalysis as fa
import yfinance as yf

# data analytic 
import numpy as np
import pandas as pd

# additional liberies
import datetime
import statistics as st

In [2]:
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [3]:
api_key = '00ef9804fcde0edd93b1b4821ee2f06a'  #job.chap@icloud.com
# api_key = 'c81352430e2fe3c941faf0814227562b'  #jobs.chaps@gmail.com
# api_key = 'a765d11740cccfb61177da0ad8699d1e'  #job.chap@gmail.com

In [6]:
''' Тикеры '''
tick = 'F'  # поменял название переменно ticker на tick, для настройки по одной акции

''' для фильтрации'''
# filt_start_date = '2000'
# filt_finish_date = '2022'


''' время '''
stime = '2000-01-01'
ftime = '2023-02-01'
period = '1mo'  # '1d' '1wk'

In [10]:
stime[:4]

'2000'

In [58]:
class Fund_ticker:
    def __init__(self, tick, stime, ftime, api_key_fund):
        self.tick = tick
        self.stime = stime
        self.ftime = ftime
        self.api_key_fund = api_key_fund
        
    def get_key_metr(self):
        ''' получаем df с key_metrics
        '''
        key_metrics = fa.key_metrics(self.tick, self.api_key_fund, period='annual')
        return key_metrics.T
    
    def get_income_statement(self):
        ''' получаем df с income_statement
        '''        
        income_statement = fa.income_statement(self.tick, self.api_key_fund, period='annual')
        return income_statement.T
        
    
    def get_balance_sheet_statement(self):
        ''' получаем df с balance_sheet_statement
        '''  
        balance_sheet_statement = fa.balance_sheet_statement(self.tick, self.api_key_fund, period='annual')

        return balance_sheet_statement.T

    def get_cash_flow_statement(self):   
        ''' получаем df с cash_flow_statement
        '''    
        cash_flow_statement = fa.cash_flow_statement(self.tick, self.api_key_fund, period='annual')
        return cash_flow_statement.T

    def get_financial_ratios(self):    
        ''' получаем df с financial_ratios
        '''
        financial_ratios = fa.financial_ratios(self.tick, self.api_key_fund, period='annual')
        return financial_ratios.T

    def get_financial_statement_growth(self):
        ''' получаем df с financial_statement_growth
        ''' 
        financial_statement_growth = fa.financial_statement_growth(self.tick, self.api_key_fund, period='annual')
        return financial_statement_growth.T
    
    
    def all_metrics(self):
        ''' конкатенация метрик со всех полученных таблиц со значениями
        '''
        df_km = self.get_key_metr()
        df_is = self.get_income_statement()
        df_bss = self.get_balance_sheet_statement()
        df_cfs = self.get_cash_flow_statement()
        df_fr = self.get_financial_ratios()
        df_fsg = self.get_financial_statement_growth()
        
        df_all_fa_columns = pd.concat([df_km,
                                       df_is,
                                       df_bss,
                                       df_cfs,
                                       df_fr,
                                       df_fsg
                                       ], axis=1)
        
        ''' убраем дубликаты колонок '''
        df_all_fa_columns = df_all_fa_columns.loc[:,~pd.concat([df_km,
                                                                df_is,
                                                                df_bss,
                                                                df_cfs,
                                                                df_fr,
                                                                df_fsg
                                                                ], axis=1).columns.duplicated()]
        
        
        ''' удаляются не используемые колонки '''
        df_all_fa_columns = df_all_fa_columns.drop(['period', 'reportedCurrency', 'cik', 'fillingDate', 'calendarYear', 'link', 'finalLink'], axis=1)        
        
        return df_all_fa_columns


        
    
    
        
    
        
    

In [59]:
x = Fund_ticker(tick, stime, ftime, api_key).all_metrics()

In [60]:
x.shape

(38, 234)